## Imports

In [2]:
import queries
import pandas as pd
from db_connection import get_sql_as_df
from df_manipulation import split_dataframe_by_column_unique_values, convert_and_match_ndc
from excel_template import create_excel_template
from pathlib import Path
import re

from datetime import date

## Query DB

In [3]:
df_rx_price = get_sql_as_df('rx_price_query')
df_rx_price.shape
# df_rx_price.head()
# df_rx_price.info()
# df_rx_price.describe()

(3162, 3)

In [4]:
# df_rx_catalog = get_sql_as_df('rx_catalog_query')
# df_rx_catalog.shape
# display(df_rx_catalog.head())
# display(df_rx_catalog.info())

In [5]:
# df_rx_distributor_list = get_sql_as_df('rx_distributor_list')

# drop "Animal Health Logistics LLC" from rx_distributor_list as it is not a required distributor
# df_rx_distributor_list = df_rx_distributor_list[df_rx_distributor_list['vendor_name'] != 'Animal Health Logistics LLC']
# df_rx_distributor_list.shape
# display(df_rx_distributor_list.head())
# display(df_rx_distributor_list.info())

In [6]:
# df_rx_catalog_applicable_vendors = df_rx_catalog[df_rx_catalog.vendor_name.isin(df_rx_distributor_list.vendor_name)]
# df_rx_catalog_applicable_vendors.shape

## Read in Vendor Response Catalogs

In [7]:
vendor_response_path = Path(r"C:\Users\isharma\OneDrive - Chewy.com, LLC\Documents\GitHub\custom_template_generator\received_rx")
vendor_response_path.exists()

True

In [189]:
rx_template_columns = ['agreement_number',
                            'Rx Number',
                            'Vendor_Number',
                            'Vendor Name',
                            'Chewy SKU',
                            'Parent Company',
                            'Brand',
                            'Item Name',
                            'Vendor Part Number',
                            'Purchasing UOM',
                            'Purchasing UOM Qty',
                            'Vendor Purchasing Cost (at UOM)',
                            'Cost per Sellable UOM \n(Vendor Purchasing Cost/Purchasing UOM Qty)',
                            'Existing Order Multiples',
                            'Existing Minimum Order Quantity',
                            'Manufacturer',
                            'NDC',
                            'Discontinued (Y/N)',
                            'New Vendor Cost Per Purchasing UOM',
                            'New Vendor Purchasing UOM',
                            'New Vendor Purchasing UOM Qty',
                            'New cost per sellable UOM\n(Calculated)',
                            'New VPN',
                            'New Manufacturer',
                            'New NDC',
                            'Updated Order Multiple',
                            'Updated Minimum Order Quantity',
                            'MBO (Manufacturer Back Ordered)\n(Y/N)',
                            'MBO Estimate Recovery Date (mm/dd/yyyy)',
                            'Current Cost per Purchasing UOM',
                            'Current Cost per Sellable UOM Quantity',
                            'New Cost per Purchasing UOM Quantity',
                            'New Cost per Sellable UOM Quantity',
                            'Existing MSRP (list)',
                            'Cost % Difference',
                            'Old Margin',
                            'New Margin',
                            'Margin Difference',
                            'Vendor Filled?',
                            'NDC Format Check']

fillable_vendor_data_columns = [ 'Discontinued (Y/N)',
                            'New Vendor Cost Per Purchasing UOM',
                            'New Vendor Purchasing UOM',
                            'New Vendor Purchasing UOM Qty',
                            # 'New cost per sellable UOM\n(Calculated)',
                            'New VPN',
                            'New Manufacturer',
                            'New NDC',
                            'Updated Order Multiple',
                            'Updated Minimum Order Quantity',
                            'MBO (Manufacturer Back Ordered)\n(Y/N)',
                            'MBO Estimate Recovery Date (mm/dd/yyyy)',
                            ]

rx_template_column_dtypes = {'agreement_number': str,
                            'Rx Number': str,
                            'Vendor_Number': str,
                            'Vendor Name': str,
                            'Chewy SKU': str,
                            'Parent Company': str,
                            'Brand': str,
                            'Item Name': str,
                            'Vendor Part Number': str,
                            'Purchasing UOM': str,
                            'Purchasing UOM Qty': str,
                            'Vendor Purchasing Cost (at UOM)': str,
                            'Cost per Sellable UOM \n(Vendor Purchasing Cost/Purchasing UOM Qty)': float,
                            'Existing Order Multiples': str,
                            'Existing Minimum Order Quantity': str,
                            'Manufacturer': str,
                            'NDC': str,
                            'Discontinued (Y/N)': str,
                            'New Vendor Cost Per Purchasing UOM': str,
                            'New Vendor Purchasing UOM': str,
                            'New Vendor Purchasing UOM Qty': str,
                            'New cost per sellable UOM\n(Calculated)': str,
                            'New VPN': str,
                            'New Manufacturer': str,
                            'New NDC': str,
                            'Updated Order Multiple': str,
                            'Updated Minimum Order Quantity': str,
                            'MBO (Manufacturer Back Ordered)\n(Y/N)': str,
                            'MBO Estimate Recovery Date (mm/dd/yyyy)': str,
                            'Current Cost per Purchasing UOM': str,
                            'Current Cost per Sellable UOM Quantity': int,
                            'New Cost per Purchasing UOM Quantity': int,
                            'New Cost per Sellable UOM Quantity': float,
                            'Existing MSRP (list)': int,
                            'Cost % Difference': float,
                            'Old Margin': float,
                            'New Margin': float,
                            'Margin Difference': float,
                            'Vendor Filled?': str,
                            'NDC Format Check': str}

In [194]:
responses_as_df = []
for path in vendor_response_path.glob('*.xlsx'):
    df_vendor_response = pd.read_excel(path, 
                                       sheet_name='Vendor_Catalog', 
                                       header=1,
                                       dtype=rx_template_column_dtypes,
                                       engine='openpyxl',
                                       na_values={'New Cost per Sellable UOM Quantity': "-",
                                                  'Cost per Sellable UOM \n(Vendor Purchasing Cost/Purchasing UOM Qty)': "-",},
                                       verbose=True,
                                    #    skiprows=2,
                                       )
    responses_as_df.append(df_vendor_response)
    print(path.stem)

demo
KB Edit rx_catalog_CARDINAL HEALTH, INC._19-04-2023


## Post Processing

In [150]:
def convert_to_11_digits_if_10_digit_ndc(value):
    # print(value)
    if value == "nan" or value == "":
        return 'New NDC not provided'
    if re.match(r'\d{5}-\d{3}-\d{2}', value):
        return value[:6] + '0' + value[6:]
    elif re.match(r'\d{5}-\d{4}-\d{1}', value):
        return value[:-1] + '0' + value[-1]
    elif re.match(r'\d{4}-\d{4}-\d{2}', value):
        return '0' + value
    elif re.match(r'\d{5}-\d{4}-\d{2}', value):
        return value
    else:
        return 'Incorrect NDC format'

In [193]:
for df in responses_as_df:
    df_post_processing = df.copy()

    # Find the columns that have been filled by the vendor and create Vendor Filled? column
    vendor_filled_mask = df_post_processing[fillable_vendor_data_columns].notnull().any(axis=1)
    df_post_processing['Vendor Filled?'] = 'No'
    df_post_processing.loc[vendor_filled_mask, 'Vendor Filled?'] = 'Yes'

    
    df_post_processing['New NDC'] = df_post_processing['New NDC'].astype(str)
    df_post_processing['Chewy SKU'] = df_post_processing['Chewy SKU'].astype(str)
    df_post_processing['NDC Format Check'] = df_post_processing[df_post_processing['Vendor Filled?']=="Yes"]['New NDC'].apply(convert_to_11_digits_if_10_digit_ndc)
    df_post_processing.sort_values(by=['Vendor Filled?'], ascending=False, inplace=True)

    # Bring in List Price
    df_post_processing = df_post_processing.merge(df_rx_price, how='left', left_on='Chewy SKU', right_on='product_part_number')
    df_post_processing["Existing MSRP (list)"] = df_post_processing.product_price_current

    # Drop unneeded columns
    df_post_processing = df_post_processing.drop(columns=['product_part_number', 'product_price_current', 'product_map_price'])

    # Only keep non-blank Vendor Name rows
    df_post_processing = df_post_processing.dropna(subset=['Vendor Name'])
    
    # df_post_processing.columns.to_list() == rx_template_columns
    date_today = date.today().strftime('%m-%d-%Y')

    create_excel_template(template_df = df_post_processing,
                        template_file_path = 'rx_template.xlsx',
                        template_password = 'mssve',
                        output_path = f'./post_process_rx/{date_today}_1/',
                        sheet_name = 'Vendor_Catalog',
                        editable_columns = [],
                        editable_cells = [],
                        start_cell = 'A3',
                        col_val_for_name = 'Vendor Name',
                        cols_to_hide = [],
                        # cols_to_hide = ['A', 'B', 'C'],
                        # cols_to_hide = ['A', 'B', 'C', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM'],
                        template_name = "rx_processed",
                        date_cell = 'G2'
                        )
    
    # display(df_post_processing['Vendor Filled?'].value_counts())
    display(df_post_processing)


No     1326
Yes       6
Name: Vendor Filled?, dtype: int64

,agreement_number,Rx Number,Vendor_Number,Vendor Name,Chewy SKU,Parent Company,Brand,Item Name,Vendor Part Number,Purchasing UOM,...,Current Cost per Sellable UOM Quantity,New Cost per Purchasing UOM Quantity,New Cost per Sellable UOM Quantity,Existing MSRP (list),Cost % Difference,Old Margin,New Margin,Margin Difference,Vendor Filled?,NDC Format Check
0,CRV000094,RX2-410-90,CRV000094,"CARDINAL HEALTH, INC.",178235,CYANOCOBALAMIN,Cyanocobalamin,Cyanocobalamin Vitamin B12 Injectable Solution...,017691,EACH,...,24,25,NaN,29.45,NaN,0.513200,NaN,NaN,Yes,New NDC not provided
1,CWVE000019,RX1-148-01,CWVE000019,"CARDINAL HEALTH, INC.",146667,FAMOTIDINE,Famotidine,"Famotidine (Generic) Tablets, 20-mg, 1 tablet",812578,BO1000,...,0,20,0.02,0.11,0.088732,0.999633,0.9996,-0.000033,Yes,Incorrect NDC format
2,RXVE000168,RX2-410-51,RXVE000168,"CARDINAL HEALTH, INC.",177775,CONFIDENCE,Confidence,"Confidence EQ for Horses, 10 gel packets",019025,EACH,...,30,0,NaN,81.88,NaN,0.394800,NaN,NaN,Yes,New NDC not provided
3,CWVE000019,RX1-148-01,CWVE000019,"CARDINAL HEALTH, INC.",146667,FAMOTIDINE,Famotidine,"Famotidine (Generic) Tablets, 20-mg, 1 tablet",800543,BO100,...,0,5,0.01,0.11,-0.767981,0.999138,0.9998,0.000662,Yes,New NDC not provided
4,CWVE000019,RX1-030-01,CWVE000019,"CARDINAL HEALTH, INC.",146525,AMANTADINE,Amantadine,"Amantadine HCl (Generic) Capsules, 100-mg, 1 c...",818131,BO100,...,0,0,NaN,0.29,NaN,0.997522,NaN,NaN,Yes,New NDC not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4602,RXVE000168,RX2-414-42,RXVE000168,"CARDINAL HEALTH, INC.",686342,NORBROOK,Midamox,"Midamox Topical Solution for Dogs, 3-9 lbs, (G...",021838,EACH,...,49,0,NaN,70.99,NaN,0.011200,NaN,NaN,No,NaN
4603,CWVE000019,RX1-171-01,CWVE000019,"CARDINAL HEALTH, INC.",146720,HYDROXYZINE,Hydroxyzine,"Hydroxyzine Pamoate (Generic) Capsules, 25-mg,...",814240,BO500,...,0,0,NaN,0.17,NaN,0.999208,NaN,NaN,No,NaN
4604,CRV000094,RX1-187-02,CRV000094,"CARDINAL HEALTH, INC.",146551,BENAZEPRIL,Benazepril,"Benazepril HCl (Generic) Tablets, 10-mg, 1 tablet",817999,BO100,...,0,0,NaN,0.14,NaN,0.999238,NaN,NaN,No,NaN
4605,CWVE000019,RX2-413-32,CWVE000019,"CARDINAL HEALTH, INC.",310478,SODIUM CHLORIDE,Sodium Chloride,"Sodium Chloride (Generic) Ophthalmic Ointment,...",818049,EACH,...,9,0,NaN,31.59,NaN,0.820000,NaN,NaN,No,NaN


## Scratch

In [72]:
# # Test with Sample data
# data = {'formats': ['9999-9999-99', '99999-999-99', '99999-9999-9', '1234-5678-90', '12345-678-90', '12345-6789-0', 'invalid-1', 'invalid-2']}
# df = pd.DataFrame(data)

# converted_df = df_manipulation.convert_and_match_ndc(df, 'formats', 'NDC Format Check')
# print(converted_df)

        formats      NDC Format Check
0  9999-9999-99         09999-9999-99
1  99999-999-99         99999-0999-99
2  99999-9999-9         99999-9999-09
3  1234-5678-90         01234-5678-90
4  12345-678-90         12345-0678-90
5  12345-6789-0         12345-6789-00
6     invalid-1  Incorrect NDC format
7     invalid-2  Incorrect NDC format


In [46]:
# df_ndc_check[df_ndc_check.converted_11_digits == 'Incorrect NDC format'].to_csv('incorrect_ndc.csv')

In [67]:
# import importlib
# import df_manipulation

# importlib.reload(df_manipulation)

<module 'df_manipulation' from 'c:\\Users\\isharma\\OneDrive - Chewy.com, LLC\\Documents\\GitHub\\edlp_audit_custom_template_generator\\df_manipulation.py'>